* LangChain: https://python.langchain.com/en/latest/
* OpenAI API: https://platform.openai.com/docs/
* FAISS: https://github.com/facebookresearch/faiss
* Pandas: https://pandas.pydata.org/docs/

In [ ]:
# !pip install -Uq openai langchain langchain-openai faiss-cpu
# !pip install -Uq langchain-community  langchain-experimental

In [ ]:
import os
import pandas as pd
import sqlite3
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_openai import OpenAI, ChatOpenAI
from langchain.memory import ConversationBufferMemory
# from langchain.agents import create_pandas_dataframe_agent
from langchain_experimental.agents import create_pandas_dataframe_agent

from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
# 데이터베이스 연결 및 데이터 로드
conn = sqlite3.connect('yes24_books_it.db')
df = pd.read_sql_query("SELECT * FROM books", conn)
conn.close()

# 제목과 설명을 결합한 필드 생성
df["title_desc"] = "제목: " + df["title"] + ", 내용: " + df["description"]

# 모든 관련 정보를 포함한 책 메타데이터 생성
book_metadata = df.apply(lambda x: f"제목: {x['title']} 저자: {x['author']} 출판사: {x['publisher']} "
                                   f"출판일: {x['pub_date']} 가격: {x['price']} 할인: {x['discount']} "
                                   f"평점: {x['rating']} 리뷰 수: {x['review_count']} 내용: {x['description']}\n\n", axis=1)


In [ ]:
# OpenAI 임베딩 초기화
embeddings = OpenAIEmbeddings()

# FAISS 벡터 저장소 생성
vector_store = FAISS.from_texts(book_metadata.tolist(), embeddings)

# 검색 및 질문-답변 체인 설정
llm = ChatOpenAI(temperature=0.2)  # 더 일관된 응답을 위해 낮은 temperature 설정
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    verbose=True
)

# AI 어시스턴트를 위한 프롬프트 템플릿
prompt_template = """당신은 IT 서적 전문가입니다. 주어진 정보를 바탕으로 사용자의 질문에 정확하고 상세하게 답변해주세요.
답변 시 책 제목, 저자, 출판사 등의 정보를 포함해주세요.
확실하지 않은 정보에 대해서는 "확실하지 않습니다"라고 말해주세요.

질문: {question}

답변: """

# 질문하고 답변을 얻는 함수
def ask_question(query):
    result = qa_chain({"question": prompt_template.format(question=query)})
    answer = result['answer']
    sources = result['source_documents']

    print("답변:", answer)
    print("\n참고한 책:")
    for i, doc in enumerate(sources, 1):
        print(f"{i}. {doc.page_content.split('내용:')[0]}")

# 사용 예시
query = "챗GPT를 활용한 책의 제목은 무엇인가요?"
ask_question(query)

# 더 복잡한 분석을 위한 Pandas DataFrame 에이전트 생성
agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df, verbose=True)

# 데이터 분석을 위한 에이전트 사용 예시
response = agent.run("평점과 리뷰 수를 분석하여 가장 인기 있는 책들의 특징을 설명해주세요. "
                     "또한, 평점과 리뷰 수 사이의 상관관계가 있다면 그에 대해서도 설명해주세요.")
print(response)